In [1]:
import sys,os
import json
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *
from transformers import set_seed

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_value = 42
set_seed(seed_value)
# supports tasks:
# cb
# rte
# sick
# mrpc
# boolq
# csqa ?????????? or mc?
# argument
# scitail
# imdb
# sst2
# qqp
# mnli

tasks = ["cb","rte","sick","mrpc","boolq","argument","scitail","imdb","sst2","qqp","mnli"]
#tasks = ["qqp","mnli"]
# task = "scitail"
model_name = "bert-base-uncased"
output_path = "C:/Users/Hector Auvinen/Desktop/eval_results/bigger_eval_steps"
adapter_conf_path = "../src/configs/adapter_configs.json"

"""output_adapter = config["output_adapter"]
mh_adapter = config["mh_adapter"]
reduction_factor = config["reduction_factor"]
non_linearity = config["non_linearity"]"""

'output_adapter = config["output_adapter"]\nmh_adapter = config["mh_adapter"]\nreduction_factor = config["reduction_factor"]\nnon_linearity = config["non_linearity"]'

In [3]:
# MOVE THIS TO MODULE
with open(adapter_conf_path, "r") as json_file:
    adapter_configs = json.load(json_file)
adapter_configs

{'output_adapter_redf_2': {'output_adapter': True,
  'mh_adapter': False,
  'reduction_factor': 2,
  'non_linearity': 'relu'},
 'output_adapter_redf_16': {'output_adapter': True,
  'mh_adapter': False,
  'reduction_factor': 16,
  'non_linearity': 'relu'},
 'output_adapter_redf_64': {'output_adapter': True,
  'mh_adapter': False,
  'reduction_factor': 64,
  'non_linearity': 'relu'}}

In [4]:
for name,config in adapter_configs.items():
    print(name,config)
    output_dir = os.path.join(output_path,name)
    
    if not os.path.exists(output_dir):
        # If the folder doesn't exist, create it
        os.makedirs(output_dir)
        print(f"Folder '{output_dir}' created.")
    else:
        print(f"Folder '{output_dir}' already exists.")
    
    print(f"**********************************RUNNING CONFIG {name}*****************************")
    print("CONFIG",config)
    for task in tasks:
        print(f"**********************************RUNNING TASK {task}*****************************")
        # load dataset
        data = load_hf_dataset(task,debug=False)
        # get tokenizer (bert)
        tokenizer = get_tokenizer(model_name)
        # get encoding method for particular task
        encode = get_encoding(task)
        # apply encoding
        dataset = preprocess_dataset(data,encode,tokenizer)
        # get label count
        num_labels = get_label_count(dataset)
        # set up model (head with num labels)
        model = setup_model(model_name,num_labels,dataset)
        
        # set up adapter config
        adapter_config = adapters.BnConfig(
                                output_adapter=config["output_adapter"],
                                mh_adapter=config["mh_adapter"],
                                reduction_factor=config["reduction_factor"],
                                non_linearity=config["non_linearity"])

        # add adapter
        model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)
        
        # set up training args
        final_output = os.path.join(output_dir,task)
        default_args = TrainingParameters(output_dir=final_output,
                                          per_device_train_batch_size=8,
                                          evaluation_strategy="epoch",
                                          eval_steps=1,
                                          save_strategy="epoch",
                                          logging_steps=200)
        default_args.lr_scheduler_type = "linear"
        train_args = get_training_arguments(default_args)
        
        # set up trainer
        trainer = get_trainer(train_args,dataset,model,early_stopping=3)
        # train
        trainer.train()
        
        # evaluate and write results to file
        eval_results = trainer.evaluate()
        write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

    

output_adapter_redf_2 {'output_adapter': True, 'mh_adapter': False, 'reduction_factor': 2, 'non_linearity': 'relu'}
Folder 'C:/Users/Hector Auvinen/Desktop/eval_results/bigger_eval_steps\output_adapter_redf_2' already exists.
**********************************RUNNING CONFIG output_adapter_redf_2*****************************
CONFIG {'output_adapter': True, 'mh_adapter': False, 'reduction_factor': 2, 'non_linearity': 'relu'}
**********************************RUNNING TASK cb*****************************
using bert tokenizer
getting encoding:
<function encode_cb at 0x000001B0B12024C0>
mapped
{'train': ['premise', 'hypothesis', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['premise', 'hypothesis', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['premise', 'hypothesis', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask']}


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                
  3%|▎         | 32/960 [00:13<04:53,  3.16it/s]

{'eval_loss': 0.817361056804657, 'eval_accuracy': 0.6607142857142857, 'eval_runtime': 1.0843, 'eval_samples_per_second': 51.646, 'eval_steps_per_second': 6.456, 'epoch': 1.0}


                                                
  7%|▋         | 64/960 [00:25<04:09,  3.60it/s]

{'eval_loss': 0.7494312524795532, 'eval_accuracy': 0.7321428571428571, 'eval_runtime': 1.1531, 'eval_samples_per_second': 48.566, 'eval_steps_per_second': 6.071, 'epoch': 2.0}


...

 15%|█▍        | 2000/13770 [15:17<1:27:04,  2.25it/s] 

{'loss': 0.2114, 'learning_rate': 8.547567175018157e-05, 'epoch': 4.36}


 16%|█▌        | 2200/13770 [16:46<1:25:05,  2.27it/s]

{'loss': 0.1939, 'learning_rate': 8.402323892519971e-05, 'epoch': 4.79}


 17%|█▋        | 2295/13770 [17:27<1:15:10,  2.54it/s]

In [5]:
##### full run:
####### MOVE THIS TO A .PY SCRIPT. ONLY HERE FOR A TEST RUN
task = "mrpc"
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

dataset = preprocess_dataset(data,encode,tokenizer)


num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"

train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

eval_results = trainer.evaluate()

write_eval_results(eval_results,output_dir,task,trainer,adapter_config)
##### full run ends. MOVE THIS.

Generating test split: 100%|██████████| 1725/1725 [00:00<00:00, 216201.24 examples/s]


using bert tokenizer
getting encoding:
<function encode_mrpc at 0x000002E00F811550>


Map: 100%|██████████| 1725/1725 [00:02<00:00, 599.53 examples/s]


mapped
{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}
labels 2


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 29/13770 [00:12<1:41:40,  2.25it/s]

KeyboardInterrupt: 

In [4]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

using bert tokenizer
getting encoding:
<function encode_scitail at 0x000002C001F6E550>


In [5]:
dataset = preprocess_dataset(data,encode,tokenizer)

Map:   0%|          | 0/23097 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not return

mapped
{'train': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask']}


In [6]:
#id2label = {id: label for (id,label) in enumerate(dataset["train"].features["labels"].names)}
#num_labels = len(id2label)
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"


labels 2


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                    
  0%|          | 50/86640 [00:45<8:09:11,  2.95it/s]

{'eval_loss': 0.5120983719825745, 'eval_accuracy': 0.7576687116564417, 'eval_runtime': 26.097, 'eval_samples_per_second': 49.967, 'eval_steps_per_second': 6.246, 'epoch': 0.02}


  0%|          | 100/86640 [01:27<7:49:31,  3.07it/s]

{'eval_loss': 0.482826828956604, 'eval_accuracy': 0.7507668711656442, 'eval_runtime': 26.8769, 'eval_samples_per_second': 48.517, 'eval_steps_per_second': 6.065, 'epoch': 0.03}


                                                       
  0%|          | 150/86640 [02:12<7:49:15,  3.07it/s]

...

                                                       
  1%|          | 650/86640 [09:45<8:40:19,  2.75it/s]

{'eval_loss': 0.3124051094055176, 'eval_accuracy': 0.8734662576687117, 'eval_runtime': 27.8346, 'eval_samples_per_second': 46.848, 'eval_steps_per_second': 5.856, 'epoch': 0.23}


  1%|          | 650/86640 [09:46<21:32:15,  1.11it/s]

{'train_runtime': 586.0818, 'train_samples_per_second': 1182.275, 'train_steps_per_second': 147.829, 'train_loss': 0.4591100810124324, 'epoch': 0.23}


TrainOutput(global_step=650, training_loss=0.4591100810124324, metrics={'train_runtime': 586.0818, 'train_samples_per_second': 1182.275, 'train_steps_per_second': 147.829, 'train_loss': 0.4591100810124324, 'epoch': 0.23})

In [8]:
eval_results = trainer.evaluate()

100%|██████████| 163/163 [00:27<00:00,  5.85it/s]


In [8]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 0.6164497137069702, 'eval_accuracy': 0.6672782874617736, 'eval_runtime': 46.1558, 'eval_samples_per_second': 70.847, 'eval_steps_per_second': 8.861, 'epoch': 0.64}


In [9]:
"""for attr_name, attr_value in vars(default_args).items():
    print(f"{attr_name}: {attr_value}")"""
"""
    train_args = {"label_names":["labels"],
    "evaluation_strategy":"steps",
    "learning_rate":1e-4,
    "num_train_epochs":1,
    "per_device_train_batch_size":8,
    "per_device_eval_batch_size":8,
    "eval_steps":50,
    "logging_steps":200,
    "output_dir":"/eval_results",
    "overwrite_output_dir":True,
    "remove_unused_columns":False,
    "lr_scheduler_type":'linear',
    "load_best_model_at_end":True,
    "metric_for_best_model" : "accuracy",
    "early_stopping_patience":3,
    "save_total_limit":5
}
"""

'\n    train_args = {"label_names":["labels"],\n    "evaluation_strategy":"steps",\n    "learning_rate":1e-4,\n    "num_train_epochs":1,\n    "per_device_train_batch_size":8,\n    "per_device_eval_batch_size":8,\n    "eval_steps":50,\n    "logging_steps":200,\n    "output_dir":"/eval_results",\n    "overwrite_output_dir":True,\n    "remove_unused_columns":False,\n    "lr_scheduler_type":\'linear\',\n    "load_best_model_at_end":True,\n    "metric_for_best_model" : "accuracy",\n    "early_stopping_patience":3,\n    "save_total_limit":5\n}\n'